In [ ]:
import boto3

In [ ]:
polly_client = boto3.client('polly', region_name='us-east-1')

In [ ]:
text = 'Hello everyone, I hope you are enjoying the course.'

In [ ]:
response = polly_client.synthesize_speech(Engine='neural',
                                        Text=text,
                                        OutputFormat='mp3',
                                        VoiceId='Matthew')

In [ ]:
with open('speech.mp3', 'wb') as file:
    file.write(response['AudioStream'].read())

In [ ]:
client = boto3.client('s3', region_name='ap-east-1')
client.create_bucket(Bucket='demo-bucket-transcription-sp', CreateBucketConfiguration={'LocationConstraint': 'ap-east-1'})

In [ ]:
client.upload_file(Filename='speech.mp3', Bucket='demo-bucket-transcription-sp', Key='speech.mp3')

In [ ]:
transcribe_client = boto3.client('transcribe', region_name='ap-east-1')
job_name = 'TranscribeJobCourse'

transcribe_client.start_transcription_job(TranscriptionJobName=job_name,
                                         Media={'MediaFileUri': 's3://demo-bucket-transcription-sp/speech.mp3'},
                                               MediaFormat='mp3',
                                               LanguageCode='en-US'
                                         )

In [ ]:
import time

while True:
    
    status = transcribe_client.get_transcription_job(TranscriptionJobName=job_name)
    if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
        break
    print('Still working on transcription')
    time.sleep(5)
print(status)

In [ ]:
import requests

response = requests.get(status['TranscriptionJob']['Transcript']['TranscriptFileUri'])
with open('transcribed.json', 'wb') as file:
    file.write(response.content)

In [ ]:
import json
with open('transcribed.json', 'r') as file:
    data = json.load(file)

In [ ]:
full_text = data['results']['transcripts'][0]['transcript']

In [ ]:
full_text

In [ ]:
transcribe_client.delete_transcription_job(TranscriptionJobName=job_name)

In [ ]:
translate = boto3.client('translate', region_name='ap-east-1')
response = translate.translate_text(Text=full_text,
                                   SourceLanguageCode='en',
                                   TargetLanguageCode='es-MX')

In [ ]:
response